In [2]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import numpy as np
import pandas as pd

In [4]:
### Load the trained model,scler pickle,one hot
model=load_model('model.h5')

## load the encoder and scaler
with open('label_encoder_gender.pkl', 'rb') as f:
    label_encoder_gender = pickle.load(f)

with open('one_hot_encoder_geo.pkl', 'rb') as f:
    one_hot_encoder_geo = pickle.load(f)

with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

In [5]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [9]:
# One-hot encode 'Geography'
geo_encoded = one_hot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=one_hot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df


b:\Data Science\KrishNaik_Udmey\ANN Classification\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [10]:
input_df=pd.DataFrame([input_data])
input_df 

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [13]:
# Encode categorical ' varib    
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])


ValueError: y contains previously unseen labels: 1

In [14]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [15]:
## concatination of one hot encoded 
input_df=pd.concat([input_df.drop('Geography', axis=1), geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [16]:
input_scaled=scaler.transform(input_df)

In [18]:
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [20]:
#Predict churn
prediction=model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 345ms/step


array([[0.02623774]], dtype=float32)

In [26]:
prediction_proba=prediction[0][0]

In [27]:
prediction_proba

0.02623774

In [28]:
if prediction_proba > 0.5:
    print("The customer is likely to churn with a probability of {:.2f}%".format(prediction_proba * 100))
else:
    print("The customer is unlikely to churn with a probability of {:.2f}%".format((1 - prediction_proba) * 100))

The customer is unlikely to churn with a probability of 97.38%
